In [ ]:
#Allows me to see what GPU the system is using .
#I required a Tesla T4 because it provided me with higher VRAM which allowed better perfromance when trianing. 
#The higher the VRAM the the higher the batch size could be.
!nvidia-smi

Mon Apr 25 21:35:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Connecting all my data which is stored in my Google Drive.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Establishing an enviroment for my models.
import os
import sys
os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/TensorFlow/models"
sys.path.append("/content/gdrive/My Drive/TensorFlow/models/research")

In [ ]:
#Clones all the models I required for training and stores them on Google Drive.
%cd '/content/gdrive/My Drive/TensorFlow'

!git clone https://github.com/tensorflow/models.git

%cd '/content/gdrive/MyDrive/TensorFlow/models'
!git checkout -f e04dafd04d69053d3733bb91d47d0d95bc2c8199

In [ ]:
#Downloads and installing all the neccessary packages needed for training, as well as updating exsiting ones.
!pip install Cython pandas tf-slim lvis
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!apt-get update --fix-missing
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools


In [ ]:
#Installs Protoc which is required to funciton with python3.
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
#Runs the setup file in my G-Drive, then proceeds to run it.
!python setup.py build
!python setup.py install


In [ ]:
#Reverts TensorFlow to an Older version so it can work with Keras.
!pip install --upgrade tensorflow-gpu==2.7.*
!pip install --upgrade tensorflow==2.7.*

In [ ]:
#Running a test to make sure my data is being read properly and all exsisting packages are working as they should be.
%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'

!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

In [ ]:
#Paths into preprocessing.
%cd '/content/gdrive/My Drive/TensorFlow/Scripts/Preprocessing'

#Allows the system to generate a TFrecord, it compresses the images into a smaller and more readable file. One for training and one for testing.
#TFrecords are required to train the model.
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/Workspace/Training_Demo/Images/Train/' -l '/content/gdrive/My Drive/TensorFlow/Workspace/Training_Demo/Annotations/label_map.pbtxt/' -o '/content/gdrive/My Drive/TensorFlow/Workspace/Training_Demo/Annotations/train.record/'
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/Workspace/Training_Demo/Images/Test/' -l '/content/gdrive/My Drive/TensorFlow/Workspace/Training_Demo/Annotations/label_map.pbtxt/' -o '/content/gdrive/My Drive/TensorFlow/Workspace/Training_Demo/Annotations/test.record/'

In [ ]:
#Paths into training_demo
%cd '/content/gdrive/My Drive/TensorFlow/Workspace/Training_Demo'
#Tensorboard allows me to see my data and the effiency of my models in real time. Useful for making adjustments when needed.
%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn

In [ ]:
#Path into training_demo
%cd '/content/gdrive/My Drive/TensorFlow/Workspace/Training_Demo'
#my_SSD_resnet model that I am currently training.
#This were the training begins.
!python model_main_tf2.py --model_dir=Models/my_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 --pipeline_config_path=Models/my_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config

In [ ]:
#Once training is complete I can begin exporting my model for evaluation.
%cd '/content/gdrive/My Drive/TensorFlow/workspace/trainingdemo'
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/ --output_directory ./exported-models/my_ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

In [ ]:
#Using completed models for testing on a new dataset.

%cd '/content/gdrive/My Drive/TensorFlow/workspace/trainingdemo'

#importing libraries
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# all the completed models for inference

#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/faster_rcnn_resnet152_v1_640x640_final/saved_model'
#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/ssd_resnet50_640x640_final/saved_model'
#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/ssd_resnet152_1024x1024_final/saved_model'
#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/faster_rcnn_640x640_final/saved_model'
#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/faster_rcnn_inception_resnet50_v2_1024x1024_final/saved_model'
#'/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/faster_rcnn_inception_resnet50_v2_640x640_final/saved_model'


#Importing all the elements needed for inference.
final_model_path = "exported-models/ssd_resnet152_final/saved_model"
loaded_model = tf.saved_model.load(final_model_path)

print("Model loading completed")

In [ ]:
#Converting Label map so its compatiable for inference.

label_map_path = '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt'

#Classifies each vehicle which present in a image.
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)

In [ ]:
Loading the test images that will be used for inference.

##Examples for images folder

# 00002.JPG - single car
# 00003.JPG - multiple vehicles
# 00074.JPG - urban street
# 00004.JPG - all for clases
# 00690_a7f65606a8_b.jpg - bike and bus
# 52343-p803xta-160906_253165604_o.JPG - two busses
# 71139-dg52vfm-jet-wincanton_203136515_o.JPG - three busses
# a543hac-stagecoach-devon_495256095_o.JPG - two vehicles
# asda-dk04rax_140476782_o.JPG - numerous cars


##Final test images 

# "images/final/00002.jpg","images/final/00003.jpg","images/final/000033.jpg","images/final/4.jpg","images/final/5.jpg","images/final/6.jpg","images/final/7.jpg","images/final/8.jpg","images/final/9.jpg","images/final/10.jpg","images/final/11.jpg","images/final/12.jpg"
# "images/final/13.JPG","images/final/14.JPG","images/final/15.JPG","images/final/16.JPG","images/final/17.JPG","images/final/18.JPG","images/final/19.JPG","images/final/20.JPG","images/final/21.JPG","images/final/22.JPG","images/final/23.JPG","images/final/24.JPG","images/final/25.JPG"


#test_images = ["images/test/00002.JPG","images/test/00003.JPG", "images/test/71139.JPG", "images/test/00004.JPG"]
test_images = ["images/final//00002.JPG","images/final/00003.JPG","images/final/71139.JPG","images/final/00004.JPG"]
print(test_images)

In [ ]:
#@title
#Runs the detection software
#Allows me to compare the performance of the final trained models to using prior results.

%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

#Imports the images for testing into a numpty array
def image_numpy_array(path):
    return np.array(Image.open(path))

for image in test_images:
  image_numpy=image_numpy_array(image)

  #The images used for testing are converted into tensors.
  tensor_convert=tf.convert_to_tensor(image_numpy)
  #Numerous images can be used at one.
  tensor_convert=tensor_convert[tf.newaxis,...]

  model_detection=loaded_model(tensor_convert)


  num_detections=int(model_detection.pop('num_detections'))
  model_detection={key:value[0,:num_detections].numpy()
                    for key,value in model_detection.items()}
  model_detection['num_detections']=num_detections

  model_detection['detection_classes']= model_detection['detection_classes'].astype(np.int64)
  
  image_np_with_detections=image_numpy.copy()

  #configuring the image that will be displayed
  viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,         # The image that will be displayed.
            model_detection['detection_boxes'],    # Displays the box for each object.
            model_detection['detection_classes'],  # Displays the objects class.
            model_detection['detection_scores'], #Displays the accuracy of each object.
            category_index,           #Utilises the label maps category.
            use_normalized_coordinates=True, # Nomalizes the coordinates so they can be analysed.
            line_thickness = 7,      # Adjusts each line thickness.
            max_boxes_to_draw=100,   #Caps the number of boxes to be displayed.
            min_score_thresh=.6,  #Boxes will only be displayed if accuracy is 60% or above.
            agnostic_mode=False)

  print('Completed Detection for {}... '.format(image), end='')


  # Once configuration is complete the image is outputted.
  %matplotlib inline
  plt.figure()

  # final image to be outputted.
  plt.figure(figsize=(40,20)) 
  plt.imshow(image_np_with_detections)
  plt.show()
  